딥러닝은 `y=Wx+b`에서 최적의 W와 b를 찾는 과정이다.

따라서 데이터에서 원하는 특징을 효과적으로 추출하기 위해선 **올바른 Weight를 정의하는 과정**이 중요하다. 데이터의 차원 변화를 쫓으며 각기 다른 신경망들이 갖는 Weight의 특성을 살펴보고, 통계&수학과 신경망 사이의 *missing link*의 진정한 의미를 알아본다.

## 데이터의 형태
딥러닝을 이해하는 방법 중 가장 쉬운 방법은 데이터의 형태 변화를 쫓는 것.
1) 10개 단어의 문장을 5개 단어로 요약했다면 정보 집약
2) 10개 단어의 문장을 20개 단어로 확장했다면 정보를 세밀하게 표현한 것.

이처럼, 1920x1080 사이즈의 흑백 이미지(1채널)는 픽셀 정보 matrix로 표현할 수 있다. 이를 컬러 이미지(3채널)로 확장시키면 픽셀 정보에 대한 matrix가 3개가 있는 것이다.
* (1920, 1080, 1) -> (1920, 1080, 3)
> 이미지 데이터는 표현 방식에 따라 (Channel, Width, Height) 또는 (Width, Height, Channel)의 방법으로 표현한다.

## Layer
하나의 물체가 여러 개의 논리적인 객체들로 구성되어 있는 경우, 이러한 각각의 객체를 하나의 레이어라 한다.

즉, 신경망은 레이어의 각기 다른 Weight들이 유기적으로 연결되어 이뤄내는 하나의 결과물이다.

### Linear Layer
Fully Connected Layer, Feedforward Neural Network, Multilayer Perceptrons, Dense Layer 등 다양한 이름으로 불리지만, 그 모든 것들은 결국 **Linear Layer**에 해당한다!!
* **선형대수학의 Linear Transform과 완전히 동일한 기능을 하는 레이어.**

위에서 말한 것과 같이 100차원으 ㅣ데이터를 300차원으로 변환한다면 데이터를 더 풍부하게 표현하는 효과가 있고, 반대로 10차원의 데이터로 변환한다면 데이터를 집약시키는 효과가 있다.

아래의 두 사각형을 가지고 데이터를 집약시키거나 늘려보자.

<img src="./assets/Layer_01.png" width=80%></img>

두 사각형 모두 (x,y) 2차원의 점 4개로 표현 가능하므로, 각각 (4,2) shape의 데이터로 표현된다. 이 두 사각형을 각각 어떤 하나의 정수로 표현해보려 한다.
* 이 정수가 우리가 구분하고자 하는 사각형의 종류, 즉 **class**이다.

#### 데이터 집약
<img src="./assets/Layer_02.png" width=90%></img>
두 사각형을, 정보가 집약된 하나의 정수로 표현하기 위해 두 단계를 거칠 수 있을 것이다.
1. (4,2) shape의 A 사각형에 *(2,1)* shape의 matrix를 선언한다. 두 matrix를 행렬곱해주면 (4,1) shape의 결과를 얻는다.
    * 여기에 곱해주기 위해 선언한 matrix를 **Weight**라고 한다.
2. 변환된 (4,1) shape의 A 사각형은 4차원이다. 이 행렬을 1차원으로 변환하는 데에는 *(4,1)* shape의 행렬이 하나 선언된다.

하지만 위 상황에서는 두 사각형에 대해 1단계 과정을 거치고 난 결과가 동일하기 때문에, 2단계 과정을 거치는 것이 의미가 없다.(입력이 동일하여 같은 결과가 나오기 때문)
* 여기서 모든 Weight의 모든 요소를 **Parameter**라고 한다.
* 위 그림에서는 2개의 Parameter가 있는 것이다.
* 만약 2단계까지 거쳤을 경우에는 6개의 파라미터겠지!!

즉, 2개의 파라미터만 가지고 이 문제를 해결하기에는 부족하다!!!

#### 데이터 늘리기
<img src="./assets/Layer_03.png" width=90%></img>
같은 방법으로 데이터를 풍부하게 만들어보자. 예를 들어
1. (4,2)에 [2x3 행렬]을 곱해서 (4,3)을 만든다.
2. (4,3)에 [3x1 행렬]을 곱해서 (4,)을 만든다.
3. (4,)에 [4x1 행렬]을 곱해서 (1,)을 만든다.

위 그림과 같이 1단계만 거쳐도 각 사각형에 대해 *독립적인 정보*가 생겨나기 시작한다. 따라서 이 방법이 아까 방법보다 더 많은 사각형을 구분해낼 수 있을 거야!!!!

데이터 집약 코드

In [2]:
# 데이터 집약
import tensorflow as tf

batch_size = 64
boxes = tf.zeros((batch_size,4,2)) #tf는 batch를 기반으로 동작.
# 사각형 2개 세트를 batch_size 개수만큼 만든 후 처리
print("1단계 연산: ", boxes.shape)

1단계 연산:  (64, 4, 2)


In [3]:
first_linear = tf.keras.layers.Dense(units=1, use_bias=False)
# unit : 출력 차원 수. 위 예제에서는 (2,1)이므로 units은 1개인것이다.

In [4]:
first_out = first_linear(boxes)
first_out = tf.squeeze(first_out, axis=-1) #(4,1)을 (4,)로 변환하는 과정. 꼭 필요한 차원 축소 과정은 아니다.
print("1단계 연산 결과:", first_out.shape)
print("1단계 Linear Layer의 Weight 모양은?", first_linear.weights[0].shape)

1단계 연산 결과: (64, 4)
1단계 Linear Layer의 Weight 모양은? (2, 1)


In [5]:
print("2단계 연산:", first_out.shape)

2단계 연산: (64, 4)


In [6]:
second_linear = tf.keras.layers.Dense(units=1, use_bias=False)
second_out = second_linear(first_out)
second_out = tf.squeeze(second_out, axis=-1)

print("2단계 연산 결과:", second_out.shape)
print("2단계 Linear Layer의 Weight 모양은!!", second_linear.weights[0].shape)

2단계 연산 결과: (64,)
2단계 Linear Layer의 Weight 모양은!! (4, 1)


데이터 늘리기 코드

In [7]:
# 데이터 늘리기
print("1단계 연산 준비:", boxes.shape)

1단계 연산 준비: (64, 4, 2)


In [8]:
fir_linear = tf.keras.layers.Dense(units=3, use_bias=False)
fir_out = fir_linear(boxes)
#fir_out = tf.squeeze(fir_out, axis=-1)

print("1단계 연산 결과:", fir_out.shape)
print("1단계 Weight의 형태:", fir_linear.weights[0].shape)

1단계 연산 결과: (64, 4, 3)
1단계 Weight의 형태: (2, 3)


In [9]:
print("2단계 연산 준비:", fir_out.shape)
sec_linear = tf.keras.layers.Dense(units=1, use_bias=False)
sec_out = sec_linear(fir_out)
sec_out = tf.squeeze(sec_out, axis=-1)

print("2단계 연산 결과:", sec_out.shape)
print("2단계 Weight shape:", sec_linear.weights[0].shape)

2단계 연산 준비: (64, 4, 3)
2단계 연산 결과: (64, 4)
2단계 Weight shape: (3, 1)


In [10]:
print("3단계 연산 준비:", sec_out.shape)

thd_linear = tf.keras.layers.Dense(units=1, use_bias=False)
thd_out = thd_linear(sec_out)
thd_out = tf.squeeze(thd_out, axis=-1)

print("3단계 연산 결과:", thd_out.shape)
print("3단계 Weight shape:", thd_linear.weights[0].shape)

3단계 연산 준비: (64, 4)
3단계 연산 결과: (64,)
3단계 Weight shape: (4, 1)


In [11]:
total_params = \
fir_linear.count_params() + \
sec_linear.count_params() + \
thd_linear.count_params()

print("총 Parameters:", total_params)

총 Parameters: 13


* 좀 더 공부해야 할 것
    * tf.squeeze
    * Dense layer의 Weight shape을 구할 때 왜 layer.weight **[0]**.shape인지

근데 파라미터가 많을수록 무조건 짱인가?

그럴리가 없지.

#### Overfitting
지나치게 많은 파라미터가 가져오는 문제.
* 너무 주어진 데이터에 엄밀하게 의존해서 새로운 데이터의 특성을 반영하지 못하는 상황.
* 주어진 데이터의 분포를 꼭 맞춰서 약간 다른 새로운 데이터가 들어왔을 때 잘 맞추지 못하는 상황.
* 학생이 시험범위 교과서 내용을 토씨 하나 틀리지 않고 다 외워서 시험을 망치는 상황.
* train set에서 아주 좋은 결과를 내지만, test set에서의 성능은 많이 떨어지는 상황.

암튼 이처럼 **Weight의 형태(모양, shape)만 선언을 해주면** 그 파라미터 값을 임의의 실수가 채우고, 수많은 데이터를 거치며 가장 적합한 Weight를 알아서 찾아가는 과정이 **Training** 이다!
* 적합한 파라미터는 주어진 데이터가 가지는 분포에 따라 결정된다.

#### Bias
아래 그림은 문화적 특성에 따른 문화 편향이다.
<img src="./assets/Layer_04.png" width=80%></img>

두 데이터의 분포는 비슷하게 생겼지만, 원점을 건들지 않고 둘을 일치시키기는 어려워 보인다! 이처럼 bias가 없다면 파라미터를 아무리 돌리고 늘리고 줄이고 해도 정확하게 근사할 수 없다.

단순히 원점을 평행이동하는 것만으로도 해결할 수 있기 때문에 실제로 bias는 **선형변환된 값에 편향 파라미터 b를 더해주는 것**으로 표현한다.

`y=Wx+b`처럼 Wx에 단순히 더하기 때문에 bias값은 상태(shape)가 *선형변환 결과 차원*인 한 줄 짜리 Weight로 정의된다.

## AutoEncoder 공부하기

## Convolution layer
Convolution filter : 이미지 입력을 처리할 때 Linear 레이어 대비 적은 파라미터로 훨씬 효과적으로 중요한 이미지 피처를 뽑아낼 수 있음.
* 이미지는 Locality가 엄청나게 중요!!!
* `Locality`란??
```
They also make strong and mostly correct assumptions about the nature of images(namely, stationarity of statistics and locality of pixel dependencies)
```
* 일단 `parameter sharing`을 알아야 함.
 
> ### Parameter sharing
    stationarity
    확률론에서 시계열의 통계적 속성이 시간에 관계없이 변하지 않은 것. 이미지는 시간 대신 위치에 관계없이 동일한 패턴들이 반복되는 특징을 가지고 있다. 따라서 stationarity 가정하에 시계열과 비슷ㅅ하게 이미지의 패턴을 파악할 수 있음. 즉, 이미지의 특정 위치에서 학습한 파라미터를 이용해서 다른 위치에 있는 동일한 특징을 추출할 수 있다는 뜻이다.
    두 명의 인물이 있는 사진에서 `입`이라는 특징을 뽑아낸다고 가정하자.
        1. 2개의 입을 하나의 특징으로 취급하여 위치와 상관없이 입 2개 추출
        2. 위치를 고려하여 서로 다른 특징 2개를 추출
    두 가지 방법 중 1번이 더 효율적이다. 위치에 상관없이 특징을 추출할 수 있기 때문. 이를 위해 파라미터를 공유하는 것이다.(후자의 경우 동일한 특징이고 위치가 다른 건데 다른 특징으로 인식해야 해서 위치마다 모두 다른 필터를 사용해야 함)
    즉, 파라미터 공유는 feature map 하나를 출력하기 위해 필터를 한 장만 유지하기 때문에 적은 파라미터 수를 사용함에도 불구하고 성능 도한 향상된다.
    입이라는 특징을 학습한 필터 하나가 이미지 전체 영역을 이동하면서(parameter sharing) Convolution 연산을 수행하면 stationarity 특성을 가진 이미지에서 한 장의 입 모양 필터로 여러개의 입 특징을 모두 추출할 수 있게 된다.
**따라서 stationarity 특성을 잘 살리면서 비교적 연산량은 더 적고 메모리 사용량도 적고, 통계적 효율성도 더 높기 때문에 CNN이 이미지 데이터를 잘 다룰 수 밖에 없즤!!!**

이미지처럼 Locality 그 자체가 엄청나게 중요한 정보가 되는 경우, Linear 레이어는 그 중요한 정보가 모두 소실된 채 엄청나게 큰 파라미터 속에서 입력과 출력 사이의 관계 가중치를 찾아야하는 무지막지한 문제를 풀어야 함. 그러나 Conv layer는 필터 구조 안에 Locality 정보가 온전히 보존됨!!

### Receptive Field
Neural Network의 출력부가 충분한 정보를 얻기 위해 커버하는 입력 데이터의 Receptive Field가 충분히 커서 그 안에 detect해야 할 object의 특성이 충분히 포함되어 있어야 정확한 detection이 가능
* Max pooling 레이어를 통해 효과적으로 Receptive Field를 키워 정보 집약 효과를 극대화할 수 있다.
* Max pooling : 필터 크기에 포함되는 영역 중 가장 큰 값을 뽑아온다.

## Deconvolution layer
Convolution layer를 통해 집약된 정보는 원본 데이터으 ㅣ정보를 잃어버린 상태. 그럼 이런 손실된 정보에 의존하는 이미지 classification이나 object detection 결과를 신뢰할 수 있을까??
* Conv layer의 조절을 통해 output shape을 늘어나게 할 수 있다.
* Upsampling : Maxpooling = downsampling이기 때문에
* Transposed Convolution

In [13]:
import numpy as np
from tensorflow.python.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.datasets import mnist
import json
import matplotlib.pyplot as plt #for plotting

# MNIST 데이터 로딩
(x_train, _), (x_test, _) = mnist.load_data()    # y_train, y_test는 사용하지 않습니다.

x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [14]:
# AutoEncoder 모델 구성 - Input 부분
input_shape = x_train.shape[1:]
input_img = Input(shape=input_shape)

# AutoEncoder 모델 구성 - Encoder 부분
encode_conv_layer_1 = Conv2D(16, (3, 3), activation='relu', padding='same')
encode_pool_layer_1 = MaxPooling2D((2, 2), padding='same')
encode_conv_layer_2 = Conv2D(8, (3, 3), activation='relu', padding='same')
encode_pool_layer_2 = MaxPooling2D((2, 2), padding='same')
encode_conv_layer_3 = Conv2D(4, (3, 3), activation='relu', padding='same')
encode_pool_layer_3 = MaxPooling2D((2, 2), padding='same')

encoded = encode_conv_layer_1(input_img)
encoded = encode_pool_layer_1(encoded)
encoded = encode_conv_layer_2(encoded)
encoded = encode_pool_layer_2(encoded)
encoded = encode_conv_layer_3(encoded)
encoded = encode_pool_layer_3(encoded)

# AutoEncoder 모델 구성 - Decoder 부분
decode_conv_layer_1 = Conv2D(4, (3, 3), activation='relu', padding='same')
decode_upsample_layer_1 = UpSampling2D((2, 2))
decode_conv_layer_2 = Conv2D(8, (3, 3), activation='relu', padding='same')
decode_upsample_layer_2 = UpSampling2D((2, 2))
decode_conv_layer_3 = Conv2D(16, (3, 3), activation='relu')
decode_upsample_layer_3 = UpSampling2D((2, 2))
decode_conv_layer_4 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')

decoded = decode_conv_layer_1(encoded)   # Decoder는 Encoder의 출력을 입력으로 받습니다.
decoded = decode_upsample_layer_1(decoded)
decoded = decode_conv_layer_2(decoded)
decoded = decode_upsample_layer_2(decoded)
decoded = decode_conv_layer_3(decoded)
decoded = decode_upsample_layer_3(decoded)
decoded = decode_conv_layer_4(decoded)

# AutoEncoder 모델 정의
autoencoder=Model(input_img, decoded)
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 4)           292       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 4)           0     

In [15]:
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.fit(x_train, x_train,
                epochs=10,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

Epoch 1/10
235/235 [==============================] - 21s 89ms/step - loss: 0.6981 - val_loss: 0.6975
Epoch 2/10
235/235 [==============================] - 10s 42ms/step - loss: 0.6970 - val_loss: 0.6964
Epoch 3/10
235/235 [==============================] - 10s 44ms/step - loss: 0.6959 - val_loss: 0.6953
Epoch 4/10
235/235 [==============================] - 11s 45ms/step - loss: 0.6947 - val_loss: 0.6941
Epoch 5/10
235/235 [==============================] - 10s 43ms/step - loss: 0.6935 - val_loss: 0.6929
Epoch 6/10
235/235 [==============================] - 11s 45ms/step - loss: 0.6923 - val_loss: 0.6917
Epoch 7/10
235/235 [==============================] - 10s 43ms/step - loss: 0.6911 - val_loss: 0.6905
Epoch 8/10
235/235 [==============================] - 10s 44ms/step - loss: 0.6898 - val_loss: 0.6892
Epoch 9/10
235/235 [==============================] - 10s 44ms/step - loss: 0.6885 - val_loss: 0.6878
Epoch 10/10
235/235 [==============================] - 11s 45ms/step - loss: 0.687

In [16]:
from tensorflow.python.keras.layers import Conv2DTranspose
# Conv2DTranspose를 활용한  AutoEncoder 모델
# AutoEncoder 모델 구성 - Input 부분
input_shape = x_train.shape[1:]
input_img = Input(shape=input_shape)

# AutoEncoder 모델 구성 - Encoder 부분
encode_conv_layer_1 = Conv2D(16, (3, 3), activation='relu')
encode_pool_layer_1 = MaxPooling2D((2, 2))
encode_conv_layer_2 = Conv2D(8, (3, 3), activation='relu')
encode_pool_layer_2 = MaxPooling2D((2, 2))
encode_conv_layer_3 = Conv2D(4, (3, 3), activation='relu')

encoded = encode_conv_layer_1(input_img)
encoded = encode_pool_layer_1(encoded)
encoded = encode_conv_layer_2(encoded)
encoded = encode_pool_layer_2(encoded)
encoded = encode_conv_layer_3(encoded)

# AutoEncoder 모델 구성 - Decoder 부분  - 
decode_conv_layer_1 = Conv2DTranspose(4, (3, 3), activation='relu', padding='same')
decode_upsample_layer_1 = UpSampling2D((2, 2))
decode_conv_layer_2 = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')
decode_upsample_layer_2 = UpSampling2D((2, 2))
decode_conv_layer_3 = Conv2DTranspose(16, (3, 3), activation='relu')
decode_upsample_layer_3 = UpSampling2D((2, 2))
decode_conv_layer_4 = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')

decoded = decode_conv_layer_1(encoded)   # Decoder는 Encoder의 출력을 입력으로 받습니다.
decoded = decode_upsample_layer_1(decoded)
decoded = decode_conv_layer_2(decoded)
decoded = decode_upsample_layer_2(decoded)
decoded = decode_conv_layer_3(decoded)
decoded = decode_upsample_layer_3(decoded)
decoded = decode_conv_layer_4(decoded)

# AutoEncoder 모델 정의
autoencoder=Model(input_img, decoded)
autoencoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 26, 26, 16)        160       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 11, 11, 8)         1160      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 8)           0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 3, 3, 4)           292       
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 3, 3, 4)           148 